In [1]:
!pip install datasets transformers evaluate langchain faiss-cpu langchain-google-genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing in

In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=dbaab59084d3e1f8ebc2f73225d24abd8fc8674ed324b227da9c3710c3229f8f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [3]:
import time
import numpy as np
import pandas as pd
import evaluate
from transformers import pipeline
from datasets import load_dataset
from datasets import concatenate_datasets
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from google.api_core.exceptions import ResourceExhausted



In [4]:
# ─── 1. Prepare 10 in-domain + 10 out-of-domain ────────────────────────────────
full_val = load_dataset("squad")["validation"]
full_train = load_dataset("squad")["train"]

in_domain = full_val.shuffle(seed=0).select(range(10))
out_domain = full_train.shuffle(seed=1).select(range(10))
eval_ds = concatenate_datasets([in_domain, out_domain])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
# ─── 2. Build index on just the in-domain contexts ────────────────────────────
docs = [Document(page_content=ex["context"], metadata={"id": str(i)})
        for i, ex in enumerate(in_domain)]
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, emb)



<ipython-input-5-c8fba1dcc3e1>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# ─── 3. Setup pipelines/chains ────────────────────────────────────────────────
# Extractive
ext_qa = pipeline("question-answering",
                  model="huggingface-course/bert-finetuned-squad",
                  tokenizer="huggingface-course/bert-finetuned-squad",
                  device=-1)



config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [8]:
# Generative (no context)
gen_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", max_tokens=64, temperature=0.0,api_key=api_key)
gen_chain = LLMChain(
    llm=gen_llm,
    prompt=PromptTemplate(
        input_variables=["question"],
        template="Answer the question: {question}"
    )
)




<ipython-input-8-5042506b2b78>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  gen_chain = LLMChain(


In [9]:
# RAG (retrieve + stuff → generative)
rag_qa = RetrievalQA.from_chain_type(
    llm=gen_llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(k=5),
)


In [10]:
# ─── 4. Load metrics ─────────────────────────────────────────────────────────
squad_metric = evaluate.load("squad")
rouge      = evaluate.load("rouge")



In [11]:
# ─── 5. Helper for retrieval metrics ─────────────────────────────────────────
def retrieval_stats(question, gold_idx):
    docs = vectorstore.as_retriever(k=5).get_relevant_documents(question)
    ids  = [int(d.metadata["id"]) for d in docs]
    # recall@5
    recall = 1.0 if gold_idx in ids else 0.0
    # mrr
    if gold_idx in ids:
        rank = ids.index(gold_idx) + 1
        mrr  = 1.0 / rank
    else:
        mrr = 0.0
    return recall, mrr



In [13]:
# ─── 6. Evaluation loop ──────────────────────────────────────────────────────
results = []
for approach, fn in [
    ("Extractive", lambda q, c, i: ext_qa(question=q, context=c)["answer"]),
    ("Generative", lambda q, c, i: gen_chain.invoke({"question": q})["text"].strip()),
    ("RAG",        lambda q, c, i: rag_qa.run(q).strip())
]:
    preds, refs = [], []
    recalls, mrrs = [], []
    times = []
    for i, ex in enumerate(eval_ds):
        q, ctx = ex["question"], ex["context"]
        golds   = ex["answers"]["text"]
        start   = time.time()
        try:
            ans = fn(q, ctx, i if i < 10 else None)
        except ResourceExhausted:
            time.sleep(60)
            ans = fn(q, ctx, i if i < 10 else None)
        elapsed = time.time() - start

        preds.append({"id": str(i), "prediction_text": ans})
        refs.append({"id": str(i), "answers": {"text": golds, "answer_start": ex["answers"]["answer_start"]}})
        times.append(elapsed)

        # only compute retrieval metrics for in-domain (i < 10)
        if approach == "RAG" and i < 10:
            r, m = retrieval_stats(q, i)
        else:
            r, m = None, None
        recalls.append(r)
        mrrs.append(m)

    # core QA metrics
    scores = squad_metric.compute(predictions=preds, references=refs)
    # rouge
    rouge_scores = rouge.compute(predictions=[p["prediction_text"] for p in preds],
                                 references=[r["answers"]["text"][0] for r in refs])
    # throughput
    total_time = sum(times)
    qps = len(times) / total_time

    results.append({
        "Approach": approach,
        "EM": scores["exact_match"],
        "F1": scores["f1"],
        "ROUGE-L": rouge_scores["rougeL"],
        "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
        "MRR@5 (in-d)":    np.nanmean([m for m in mrrs    if m is not None]),
        "Throughput (QPS)": round(qps, 2)
    })



<ipython-input-13-c4ebb902aa13>:48: RuntimeWarning: Mean of empty slice
  "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
<ipython-input-13-c4ebb902aa13>:49: RuntimeWarning: Mean of empty slice
  "MRR@5 (in-d)":    np.nanmean([m for m in mrrs    if m is not None]),
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
].
<ipython-input-13-c4ebb902aa13>:48: RuntimeWarning: Mean of empty slice
  "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
<ipython-input-13-c4ebb902aa13>:49: RuntimeWarning: Mean of empty slice
  "MRR@5 (in-d)":    np.na

In [14]:
# ─── 7. Display final table ──────────────────────────────────────────────────
df = pd.DataFrame(results)
df.set_index("Approach", inplace=True)
df


,EM,F1,ROUGE-L,Recall@5 (in-d),MRR@5 (in-d),Throughput (QPS)
Approach,,,,,,
Extractive,90.0,96.285714,0.935556,NaN,NaN,2.49
Generative,0.0,9.178409,0.090222,NaN,NaN,0.27
RAG,0.0,11.625609,0.112969,1.0,1.0,0.05
